# Point Cloud Registration


In [1]:
import utils
import open3d as o3d
import numpy as np

In [2]:
def pcr():
    transform = utils.readData("01.txt")
    lidar_to_cam = np.array([[0, 0, 1], [-1, 0, 0], [0, -1, 0]]).T
    pcdf = o3d.geometry.PointCloud()
    for i in range(10,87):
        tr = transform[i, :].reshape((3, 4))
        pc = np.matmul(lidar_to_cam,utils.readPointCloud("01/%06d.bin" % i)[:, :3].T).T
        pad = np.array([[0, 0, 0, 1.0]])
        tf = np.append(tr,pad, axis = 0) 
        ones = np.ones((pc.shape[0], 1))
        pc = np.matmul(tf, np.append(pc, ones, axis = 1).T).T
        pcreshape = pc[:, 3].reshape(-1, 1) 
        pc = pc[:, :3] / pcreshape
        pcd = o3d.geometry.PointCloud() 
        pcd.points = o3d.utility.Vector3dVector(pc)
    #     pcd_down = pcd.voxel_down_sample(voxel_size = 5) 
        pcdf += pcd
    return pcdf
pcdf = pcr()
#Registering the point clouds from the LiDAR bins to a single coordinate frame according to odometry data.

In [3]:
o3d.visualization.draw_geometries([pcdf])
o3d.io.write_point_cloud("./pcd.ply",pcdf)

True